In [5]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNNV2, RNN, RNNV12
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = './train_log/rnn_v2/1conv-0aconv/'
logger.set_logger_dir(log_dir, action='d')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1102 09:49:26 @logger.py:74] Argv: /home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1007/jupyter/kernel-f2d6d34e-e919-4aed-acf1-f7d068e9f2ab.json


In [6]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

In [7]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [8]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.stages = [6]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (1178, 6583), 'train': (1057, 6116), 'val': (234, 1328)}

In [9]:
dm.get_imbalance_ratio()

,train,val,test
dorsal prothoracic pharyngeal muscle,5.074713,4.571429,5.299465
embryonic brain,1.936111,1.752941,1.923077
embryonic central nervous system,3.194444,2.836066,3.513410
embryonic dorsal epidermis,2.815884,2.774194,2.993220
embryonic head epidermis,4.622340,4.571429,4.890000
embryonic midgut,1.076621,0.983051,1.073944
embryonic ventral epidermis,3.244980,3.333333,3.530769
embryonic/larval muscle system,3.034351,3.105263,3.104530
ubiquitous,5.181287,5.000000,5.333333
ventral nerve cord,1.849057,1.629213,1.852300


In [10]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.7
config.gamma = 2
config.use_glimpse = False
config.doubly_stochastic_lambda = 0
config.read_time = 5
config.batch_size = 64

threshold = 0.4
train_data = dm.get_train_stream()
val_data = dm.get_train_stream()

In [14]:
np.set_printoptions(formatter={'float_kind': lambda x: '%.2f' % x})
model = RNN(config, is_finetuning=False, 
              label_scale=dm.get_imbalance_ratio().train.values)
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=20, tower=[1])
Trainer(train_config).train()

[1102 09:55:09 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 16
[1102 09:55:10 @input_source.py:178] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1102 09:55:10 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:1...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1102 09:55:14 @model_utils.py:47] Model Parameters: 
name                               shape    

[1102 09:55:19 @base.py:212] Creating the session ...
[1102 09:55:21 @base.py:216] Initializing the session ...
[1102 09:55:21 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1102 09:55:23 @base.py:223] Graph Finalized.
[1102 09:55:23 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1102 09:55:23 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1102 09:55:24 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1102 09:55:24 @base.py:257] Start Epoch 1 ...


100%|##########|16/16[00:40<00:00, 0.50it/s]

[1102 09:56:05 @base.py:267] Epoch 1 (global_step 16) finished, time:40.06 sec.



  0%|          |0/16[00:00<?,?it/s]/home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|16/16[00:23<00:00, 0.65it/s]


...

[1102 10:12:39 @saver.py:89] Model saved to ./train_log/rnn_v2/1conv-0aconv/model-304.
[1102 10:12:40 @saver.py:158] Model with maximum 'micro_auc' saved.
[1102 10:12:40 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 49.665
[1102 10:12:40 @monitor.py:361] QueueInput/queue_size_1: 49.932
[1102 10:12:40 @monitor.py:361] coverage: 3.8301
[1102 10:12:40 @monitor.py:361] learning_rate: 1e-05
[1102 10:12:40 @monitor.py:361] loss/value: 0.046396
[1102 10:12:40 @monitor.py:361] macro_auc: 0.9345
[1102 10:12:40 @monitor.py:361] macro_f1: 0.59859
[1102 10:12:40 @monitor.py:361] mean_average_precision: 0.85324
[1102 10:12:40 @monitor.py:361] micro_auc: 0.85144
[1102 10:12:40 @monitor.py:361] micro_f1: 0.60039
[1102 10:12:40 @monitor.py:361] one_error: 0.24414
[1102 10:12:40 @monitor.py:361] ranking_loss: 0.17556
[1102 10:12:40 @monitor.py:361] ranking_mean_average_precision: 0.75285
[1102 10:12:40 @monitor.py:361] training_ap: 0.84064
[1102 10:12:40 @monitor.py:361] trainin

100%|##########|16/16[00:29<00:00, 0.56it/s]

[1102 10:13:10 @base.py:267] Epoch 20 (global_step 320) finished, time:29.29 sec.



100%|##########|16/16[00:22<00:00, 0.69it/s]


[1102 10:13:33 @saver.py:89] Model saved to ./train_log/rnn_v2/1conv-0aconv/model-320.
[1102 10:13:34 @saver.py:158] Model with maximum 'micro_auc' saved.
[1102 10:13:34 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 49.453
[1102 10:13:34 @monitor.py:361] QueueInput/queue_size_1: 49.523
[1102 10:13:34 @monitor.py:361] coverage: 3.5859
[1102 10:13:34 @monitor.py:361] learning_rate: 1e-05
[1102 10:13:34 @monitor.py:361] loss/value: 0.045761
[1102 10:13:34 @monitor.py:361] macro_auc: 0.93903
[1102 10:13:34 @monitor.py:361] macro_f1: 0.62433
[1102 10:13:34 @monitor.py:361] mean_average_precision: 0.86014
[1102 10:13:34 @monitor.py:361] micro_auc: 0.87231
[1102 10:13:34 @monitor.py:361] micro_f1: 0.62473
[1102 10:13:34 @monitor.py:361] one_error: 0.22559
[1102 10:13:34 @monitor.py:361] ranking_loss: 0.15499
[1102 10:13:34 @monitor.py:361] ranking_mean_average_precision: 0.77851
[1102 10:13:34 @monitor.py:361] training_ap: 0.84759
[1102 10:13:34 @monitor.py:361] traini

In [14]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'EMA/QueueInput/queue_size:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/biased:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/local_step:0' shape=() dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bot